In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier

from tianchi.user_location_prediction.pysrc.feature import *
from tianchi.user_location_prediction.pysrc.load import *

In [8]:
sample_i = load_sample_test()
sample_wifi_i = load_sample_wifi_test()
shop_i = load_shop_test()

In [ ]:
def construct_batch(sample_df, sample_wifi_df, shop_df):
    feat_list = []

    weekday_feat = weekday_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(weekday_feat)

    hour_feat = hour_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(hour_feat)

    spaceloc_feat = spaceloc_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_feat)

    spaceloc_dist_feat = spaceloc_dist_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_dist_feat)

    wifi_power_feat = wifi_power_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_power_feat)

    wifi_powerloc_dist_feat = wifi_powerloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_power_feat)
    feat_list.append(wifi_powerloc_dist_feat)

    wifi_flag_feat = wifi_flag_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_flag_feat)

    wifi_flagloc_dist_feat = wifi_flagloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_flag_feat)
    feat_list.append(wifi_flagloc_dist_feat)

    feat = np.concatenate(feat_list, axis=1)
    train_idx = sample_df[sample_df.row_id.isnull()].index.tolist()[-1] + 1

    train_x = feat[:train_idx]
    train_y = sample_df[sample_df.row_id.isnull()]['shop_id'].values
    test_x = feat[train_idx:]
    test_row_id = list(sample_df[-sample_df.row_id.isnull()]['row_id'].values)

    return train_x, train_y, test_x, test_row_id